# Overview

This notebook presents step-by-step the algorithm used for hierarchical clustering of local atomic environments.

In this example, some snapshots from a single MD trajectory are taken to consturct the hierarchical clustering. Usually, clearer results are obtained when multiple MD trajectories, containing structures (nanoparticles) with different numbers of atoms, are used.

In [1]:
from raffy import trajectory_cluster as tc
from sklearn.cluster import KMeans
from ase.io import read, write
from ase.io.extxyz import write_extxyz
import numpy as np
import json

def get_labels(vects, clusters):
    means = np.array(clusters['means'])
    stds = np.array(clusters['stds'])
    a = np.sum(((vects[:, None, :] - means[None, :, :])/stds[None, :, :])**2, axis = 2)
    labels = np.argmin(a, axis = 1)
    return labels

np.random.seed(42)

/home/claudio/postdoc/venv/raffy/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


## Compute all Local Atomic Environment Descriptors

In [2]:
# This can take a while (~30 minutes for this example of a MD trajectory of Au nanoparticle containing >2800 atoms)
frames = ':'  # Indicate which frames to analyse.
k = 4  # Number of clusters to be found
filename = "data/Au/example.xyz" # The trajectory file can be in .xyz or .dump format
cut = 4.42  # Cutoff in Angstrom of the descriptor. If not specified it is automatically set

tc.trajectory_cluster(filename, index = frames, k = k, cut=cut)

Finished. The Labeled trajectory file          can be found at data/Au/example_clustered_k=4_cut=4.42.xyz


## Create subset of descriptors to use for hierarchical clustering

In [4]:
Y = np.load("data/Au/example_G_cut_4.42.npy")[:, :-1]
n = 10000
ind = np.random.choice(np.arange(len(Y)), n, replace = False)
ind_n = ind[:n]

X = Y[ind, :]

## First level of clustering (separate inner from surface atoms)

In [5]:
k=2
kmeans = KMeans(n_clusters=k).fit(X)
labels = kmeans.predict(Y)

# Since labels 0 and 1 are randomly assigned, here we enforce 0 to be the ``inner'' label heuristically
if Y[labels == 0].mean(axis = 0)[4] < Y[labels == 1].mean(axis = 0)[4]:
    print("Inverting Labels")
    labels = 1 - labels

Inverting Labels


## Second level of clustering (separate high- from low-coordination surface atoms)

In [6]:
kmeans_surface = KMeans(n_clusters=k).fit(X[labels[ind_n] == 1])
labels_surface = kmeans_surface.predict(Y[labels == 1])

# Check the correct ordering of HS-LS atoms based on 4th component of gvector
if Y[labels == 1][labels_surface == 0].mean(axis = 0)[4] < Y[labels == 1][labels_surface == 1].mean(axis = 0)[4]:
    print('Inverting high- and low-coordination surface labels')
    labels_surface = 1 - labels_surface
    
# Label values are: 0: Solid inner, 1: Liquid inner, 2: Solid high-coord. surface, 3: Liquid high-coord surface,
# 4: Solid low-coord. surface, 5: Liquid low-coord surface, 4: 
labels *=2
labels[labels == 2] += 2*labels_surface

## Third level of clustering (separate liquid from solid atoms)

In [7]:
# Solid from liquid inner
kmeans_bulk = KMeans(n_clusters=k).fit(X[labels[ind_n] == 0])
labels_bulk = kmeans_bulk.predict(Y[labels == 0])
# Check the correct ordering of liquid-solid inner atoms based on 1st component of gvector
if Y[labels == 0][labels_bulk == 0].mean(axis = 0)[4] < Y[labels == 0][labels_bulk == 1].mean(axis = 0)[4]:
    print('Inverting bulk solid and bulk liquid labels')
    labels_bulk = 1 - labels_bulk
labels[labels==0] += labels_bulk
    
# Solid from liquid high-coordination surface
kmeans_surface_hc = KMeans(n_clusters=k).fit(X[labels[ind_n] == 2])
labels_surface_hc = kmeans_surface_hc.predict(Y[labels == 2])
# Check the correct ordering of liquid-solid HS atoms on 1st component of gvector
if Y[labels == 2][labels_surface_hc == 0].mean(axis = 0)[4] < Y[labels == 2][labels_surface_hc == 1].mean(axis = 0)[4]:
    print('Inverting high- and low-coordination surface labels')
    labels_surface_hc = 1 - labels_surface_hc
labels[labels==2] += labels_surface_hc

# Solid from liquid low-coordination surface
kmeans_surface_lc = KMeans(n_clusters=k).fit(X[labels[ind_n] == 4])
labels_surface_lc = kmeans_surface_hc.predict(Y[labels == 4])

# Check the correct ordering of liquid-solid LS atoms based on 1st component of gvector
if Y[labels == 4][labels_surface_lc == 0].mean(axis = 0)[4] < Y[labels == 4][labels_surface_lc == 1].mean(axis = 0)[4]:
    print('Inverting high- and low-coordination surface labels')
    labels_surface_lc = 1 - labels_surface_lc
labels[labels==4] += labels_surface_lc

## Store mean and standard deviation of the centroid of each cluster

In [8]:
k = 6
m = np.zeros((k, X.shape[1]))
s = np.zeros((k, X.shape[1]))

for i in np.arange(k):
    m[i] = np.mean(Y[labels == i], axis = 0)
    s[i] = np.std(Y[labels == i], axis = 0)

clusters = {}
clusters['means'] = m
clusters['stds'] = s

## Read traj and descriptors and cluster based on the centroids of the classes found

In [10]:
gvect = np.load(f"data/Au/example_G_cut_4.42.npy", allow_pickle = True)[:, :-1]
traj = read(f"data/Au/example_clustered_k=4_cut=4.42.xyz", index = ':')
labels = get_labels(gvect, clusters)

last = 0
for i in np.arange(len(traj)):
    nat = len(traj[0].get_atomic_numbers())
    if i == 0:
        traj[i].set_tags(labels[0:nat])
    else:
        traj[i].set_tags(labels[last:last + nat])
    last += nat

# Write output
write_extxyz(f"data/Au/example_labeled_cut=4.42.xyz", traj, columns=[
             'symbols', 'positions', 'tags'])